# Imports

In [1]:
import cv2 as cv
import numpy as np
from numpy.fft import fft2, fftshift, ifft2

# Leemos la imagen

Leemos en blanco y negro una imagen y le aplicamos un downsample de forma de que entre en pantalla

In [2]:
tun = cv.imread('../data/Imagenes para el TP/tun.jpg', cv.IMREAD_GRAYSCALE)
tun = cv.resize(src=tun, dsize=None, fx=1/2, fy=1/2, interpolation=cv.INTER_AREA)

Visualizamos la imagen y nos damos cuenta que en las zonas oscuras no se detectan detalles como los bordes de las rocas que conforman el techo.

In [3]:
cv.imshow(winname='Tunel', mat=tun)
cv.waitKey(0)

-1

# Filtrado Homomórfico

La imagen f(x, y) se puede expresar como $f(x, y) = i(x, y)\cdot r(x, y)$, donde $i(x, y)$ es la ilumninancia y $r(x, y)$ es la reflectancia.

Mientras que $i(x, y)$ está asociada con variaciones espaciales lentas en la imagen, $r(x, y)$ está asociada con variaciones espaciales abruptas en la imagen. Se pretende entonces operar sobre la imagen de forma de amplicar una componente y atenuar en simultáneo a otra, de forma tal de hacer prevalecer los bordes o las suavidades de una imagen.

Dado que un producto puede ser descompuesto como una suma al aplicarle un logaritmo $ln(i(x, y)\cdot r(x, y)) = ln(i(x, y)) + ln(r(x, y)$, y dado que la transformada de Fourier es una transformación lineal, se pretende hacer uso de esta propiedad realizando una nueva transformación, la denominada homomórfica, en la que el filtrado en frecuencia de cada componente se produzca luego de aplicar el logaritmo a la imagen, para tener a estas dos componentes por separado. De forma matemática:

$F(ln(i(x, y)\cdot r(x, y)))(f) = F(ln(i(x, y)))(f) + F(ln(r(x, y))(f)$

$F(ln(i(x, y)\cdot r(x, y)))(f) = F(ln(i(x, y)))(f) + F(ln(r(x, y))(f)$

Se aplican entonces los siguientes pasos para realizar:

1) Se aplica el logaritmo a cada píxel de la imagen, luego se calcula su Transformada de Fourier.

2) Se aplica el filtro deseado en el dominio de la frecuencia. En este caso el aplicado filtro a aplicar es un filtro pasa-altos gaussiano por su facilidad de implementación usando OpenCV y porque queremos resaltar las frecuencias altas del logaritmo, aquellas que resaltan los detalles de la imagen (detalles de la roca en zonas oscuras).

3) Se vuelve al dominio del espacio y se realiza la exponencial del resultado de forma de "invertir" el logaritmo.

In [4]:
def homorphic(img, apply_filter=False, sigma=1):
    sigma = sigma
    img = img.astype(dtype=np.uint16)
    img = np.log(1 + img)

    img = fftshift(fft2(img))
    img = img / np.max(np.abs(img))
    if apply_filter:
        
        kernel_x = cv.getGaussianKernel(ksize=img.shape[0], sigma=sigma, ktype=cv.CV_64F)
        kernel_y = cv.getGaussianKernel(ksize=img.shape[1], sigma=sigma, ktype=cv.CV_64F)
        kernel = np.multiply(kernel_x, np.transpose(kernel_y))
        kernel = kernel / np.max(kernel)
        kernel = 1 - kernel

        new_img = img * kernel
    else :
        new_img = img
        
    new_img = np.abs(ifft2(new_img)).astype(np.float)
    new_img = new_img / np.max(new_img)
    
    if apply_filter:  
        new_img = np.exp(new_img)
        new_img /= np.e
    
    new_img = new_img * 255
    new_img = new_img.astype(dtype=np.uint8)

    return new_img

# Resultados

Aplicamos la transformación homomórfica sin filtrar:

In [5]:
tuneado = homorphic(img=tun, apply_filter=False)
result = np.hstack((tun, tuneado))
cv.imshow(winname='No Tuneada vs. Tuneada sin filtro', mat=result)
cv.waitKey(0)

-1

Aplicamos la transformación homomórfica filtrando con un pasa-altos gaussiano y observamos los resultados:

In [6]:
tuneado = homorphic(img=tun, apply_filter=True, sigma=1)
result = np.hstack((tun, tuneado))
cv.imshow(winname='No Tuneada vs. Tuneada con filtro', mat=result)
cv.waitKey(0)

-1